# Clonamos el repositorio con los modelos y herramientas

In [1]:
!git clone https://github.com/dannasalazar11/Msc_thesis.git

Cloning into 'Msc_thesis'...
remote: Enumerating objects: 318, done.
remote: Counting objects: 100% (146/146), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 318 (delta 97), reused 52 (delta 26), pack-reused 172 (from 1)
Receiving objects: 100% (318/318), 49.93 MiB | 40.48 MiB/s, done.
Resolving deltas: 100% (188/188), done.


In [2]:
import sys
sys.path.append('/kaggle/working/Msc_thesis')

from gmrrnet_adhd.utils import get_segmented_data, train_LOSO
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')


import tensorflow as tf
import numpy as np
import random
import os

# Establecer semilla
seed = 42

# Semillas para módulos principales
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

2025-04-30 23:04:54.138520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746054294.356490      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746054294.415909      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Importar base de datos segmentada (Segmentos de 4 seg con translape del 50%, es decir, de 2 seg)

In [3]:
X, y, sbjs = get_segmented_data()
X.shape, y.shape, len(sbjs)

((8213, 19, 512), (8213, 2), 8213)

# Importamos el modelo y definimos los hiperparámetros

In [4]:
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from gmrrnet_adhd.models.cnn_lstm_eegnet import cnn_lstm_eegnet

model_name = 'CNN_LSTM_EEGNet'
model_args = {
    "n_channels": 19,      # EEG electrodes
    "n_times": 512,        # time points per trial
    "n_classes": 2,        # binary task
    "temporal_kernel": 25, # ≈200 ms at 128 Hz
    "pool_size": 20,
    "pool_stride": 10
}

compile_args = {
    'loss': CategoricalCrossentropy(),
    'optimizer': lambda: Adam(1e-2),  # función que retorna un nuevo optimizador
    'metrics': ['categorical_accuracy']
}


model = cnn_lstm_eegnet(**model_args)

model.summary()

I0000 00:00:1746054310.298059      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746054310.298751      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "cnn_lstm_eegnet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 19, 512, 1)     │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast (Cast)               │ (None, 19, 512, 1)     │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 19, 512, 50)    │          1,250 │ cast[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 19, 512, 50)    │            200 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 1, 512, 100)    │              0 │ batch_normalization[0… │
│                           │                        │                │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ depthwise_conv2d          │ (None, 1, 512, 100)    │          1,900 │ activation[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 1, 512, 100)    │            400 │ depthwise_conv2d[0][0] │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ average_pooling2d         │ (None, 1, 50, 100)     │              0 │ activation[1][0]       │
│ (AveragePooling2D)        │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 1, 50, 100)     │              0 │ average_pooling2d[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ permute (Permute)         │ (None, 50, 1, 100)     │              0 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed          │ (None, 50, 100)        │              0 │ permute[0][0]          │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 50, 10)         │          4,440 │ time_distributed[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 10)             │            840 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 2)              │             22 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 9,052 (35.36 KB)

 Trainable params: 8,752 (34.19 KB)

 Non-trainable params: 300 (1.17 KB)

# Resultados - Leave One Subject Out

In [5]:
results = train_LOSO(cnn_lstm_eegnet, X, y, sbjs, model_args, compile_args, 61, 120, model_name)

Evaluando modelo para el sujeto #61: v250


I0000 00:00:1746054319.403531      73 cuda_dnn.cc:529] Loaded cuDNN version 90300


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 703ms/step
Métricas para v250: {'accuracy': 1.0}

Evaluando modelo para el sujeto #62: v254
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 688ms/step
Métricas para v254: {'accuracy': 1.0}

Evaluando modelo para el sujeto #63: v25p
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 691ms/step
Métricas para v25p: {'accuracy': 1.0}

Evaluando modelo para el sujeto #64: v263
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 335ms/step
Métricas para v263: {'accuracy': 1.0}

Evaluando modelo para el sujeto #65: v265
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 556ms/step
Métricas para v265: {'accuracy': 1.0}

Evaluando modelo para el sujeto #66: v270
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 357ms/step
Métricas para v270: {'accuracy': 1.0}

Evaluando modelo para el sujeto #67: v274
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 337ms/step
Métricas para v274: {'accuracy': 1.0}

Evaluando modelo para el sujeto #68: v279
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 596ms/step
Métricas para v279: {'accuracy': 1.0}

Evaluando modelo para el sujeto #69: v27p
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 234ms/step
M

In [6]:
# Extraer todos los valores de accuracy
accuracies = [v['accuracy'] for v in results.values()]

# Calcular el promedio
average_accuracy = sum(accuracies) / len(accuracies)

print(f"Accuracy promedio: {average_accuracy}")

Accuracy promedio: 0.9452821892477302


In [7]:
import pickle

with open(f'results_LOSO_{model_name}.pkl', 'wb') as f:
    pickle.dump(results, f)